In [6]:
import os
import numpy as np
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import pandas as pd
import torch.nn.functional as F
from PIL import Image
import matplotlib.pyplot as plt

In [7]:
# load the model
import models
# default settings
arch = 'resnet32'
num_classes = 10
use_norm = False
model = models.__dict__[arch](num_classes=num_classes, use_norm=use_norm)

In [8]:
# load the model parameters
model_ckpt = '/vol/biomedic3/zl9518/ModelEvaluation/LDAM-DRW/checkpoint/cifar10_resnet32_CE_None_exp_0.01_0/ckpt.pth.tar'
gpu = 1
model = model.cuda(gpu)
checkpoint = torch.load(model_ckpt, map_location = 'cuda:' + str(gpu))
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [17]:
# prepare the test data.
transform_val = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
val_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_val)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False,
        num_workers=4, pin_memory=True)

Files already downloaded and verified


In [18]:
cols_names_classes = ['class_' + str(i) for i in range(0,num_classes)]
cols_names_logits = ['logit_' + str(i) for i in range(0, num_classes)]
cols_names_targets = ['target_' + str(i) for i in range(0, num_classes)]

In [19]:
cifarresultsdir = '/vol/biomedic3/zl9518/ModelEvaluation/LDAM-DRW/cifar10results/'
model.eval()
logits = []
preds = []
targets = []
for i, (input, target) in enumerate(tqdm(val_loader)):
    input = input.cuda(gpu, non_blocking = True)
    logits_test = model(input)
    preds_test = F.softmax(logits_test, dim = 1)
    targets_test = F.one_hot(target, num_classes = num_classes)
    logits.append(logits_test.cpu().detach())
    preds.append(preds_test.cpu().detach())
    targets.append(targets_test)
    
logits = torch.cat(logits, dim=0)
preds = torch.cat(preds, dim=0)
targets = torch.cat(targets, dim=0)
    
df = pd.DataFrame(data=preds.numpy(), columns=cols_names_classes)
df_logits = pd.DataFrame(data=logits.numpy(), columns=cols_names_logits)
df_targets = pd.DataFrame(data=targets.numpy(), columns=cols_names_targets)
df = pd.concat([df, df_logits, df_targets], axis=1)
df.to_csv(os.path.join(cifarresultsdir, 'predictions_val.csv'), index=False)

100%|█████████████████████| 10000/10000 [00:56<00:00, 176.91it/s]


In [20]:
# get the results on training dataset.

In [21]:
# prepare the training data.
transform_val = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
from imbalance_cifar import IMBALANCECIFAR10
val_dataset = IMBALANCECIFAR10(root='./data', imb_type='exp', imb_factor=0.01, rand_number=0, train=True, download=True, transform=transform_val)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False,
        num_workers=4, pin_memory=True)


Files already downloaded and verified


In [22]:
cols_names_classes = ['class_' + str(i) for i in range(0,num_classes)]
cols_names_logits = ['logit_' + str(i) for i in range(0, num_classes)]
cols_names_targets = ['target_' + str(i) for i in range(0, num_classes)]

In [23]:
cifarresultsdir = '/vol/biomedic3/zl9518/ModelEvaluation/LDAM-DRW/cifar10results/'
model.eval()
logits = []
preds = []
targets = []
for i, (input, target) in enumerate(tqdm(val_loader)):
    input = input.cuda(gpu, non_blocking = True)
    logits_test = model(input)
    preds_test = F.softmax(logits_test, dim = 1)
    targets_test = F.one_hot(target, num_classes = num_classes)
    logits.append(logits_test.cpu().detach())
    preds.append(preds_test.cpu().detach())
    targets.append(targets_test)
    
logits = torch.cat(logits, dim=0)
preds = torch.cat(preds, dim=0)
targets = torch.cat(targets, dim=0)
    
df = pd.DataFrame(data=preds.numpy(), columns=cols_names_classes)
df_logits = pd.DataFrame(data=logits.numpy(), columns=cols_names_logits)
df_targets = pd.DataFrame(data=targets.numpy(), columns=cols_names_targets)
df = pd.concat([df, df_logits, df_targets], axis=1)
df.to_csv(os.path.join(cifarresultsdir, 'predictions_train.csv'), index=False)

100%|█████████████████████| 12406/12406 [01:29<00:00, 137.96it/s]


In [8]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = torch.LongTensor(targets)
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        
        if self.transform:
            x = Image.fromarray(self.data[index])
            x = self.transform(x)
        
        return x, y
    
    def __len__(self):
        return len(self.data)

In [115]:
cifarresultsdir = '/vol/biomedic3/zl9518/ModelEvaluation/LDAM-DRW/cifar10results/'
cifarcdatadir = '/vol/biomedic3/zl9518/ModelEvaluation/LDAM-DRW/CIFAR-10-C/'

alltestfiles = os.listdir(cifarcdatadir)
alltestfiles.remove('labels.npy')

labelcifar = np.load(cifarcdatadir + 'labels.npy')

# for testc in tqdm(alltestfiles):
for testc in alltestfiles:
    
    print('Processing corruption: ', testc[:-4])
    
    datasetexample = np.load(cifarcdatadir + testc)
    
    val_datasetc = MyDataset(datasetexample, labelcifar, transform=transform_val)
    val_loaderc = torch.utils.data.DataLoader(val_datasetc, batch_size=1, shuffle=False,
        num_workers=4, pin_memory=True)
    
    savefilename = 'predictions_val_' + testc[:-4] + '.csv'
    
    model.eval()
    logits = []
    preds = []
    targets = []
    for i, (input, target) in enumerate(tqdm(val_loaderc)):
        input = input.cuda(gpu, non_blocking = True)
        logits_test = model(input)
        preds_test = F.softmax(logits_test, dim = 1)
        targets_test = F.one_hot(target, num_classes = num_classes)
        logits.append(logits_test.cpu().detach())
        preds.append(preds_test.cpu().detach())
        targets.append(targets_test)

    logits = torch.cat(logits, dim=0)
    preds = torch.cat(preds, dim=0)
    targets = torch.cat(targets, dim=0)

    df = pd.DataFrame(data=preds.numpy(), columns=cols_names_classes)
    df_logits = pd.DataFrame(data=logits.numpy(), columns=cols_names_logits)
    df_targets = pd.DataFrame(data=targets.numpy(), columns=cols_names_targets)
    df = pd.concat([df, df_logits, df_targets], axis=1)
    df.to_csv(os.path.join(cifarresultsdir, savefilename), index=False)
    

  0%|          | 0/50000 [00:00<?, ?it/s]

Processing corruption:  saturate


  0%|          | 0/50000 [00:00<?, ?it/s]

Processing corruption:  gaussian_blur


  0%|          | 0/50000 [00:00<?, ?it/s]

...

  0%|          | 0/50000 [00:00<?, ?it/s]

Processing corruption:  elastic_transform


100%|██████████| 50000/50000 [07:20<00:00, 113.44it/s]


Processing corruption:  defocus_blur


100%|██████████| 50000/50000 [07:19<00:00, 113.66it/s]


In [7]:
# test cifar-10-c
cifarresultsdir = '/vol/biomedic3/zl9518/ModelEvaluation/LDAM-DRW/cifar10results/'
cifarcdatadir = '/vol/biomedic3/zl9518/ModelEvaluation/LDAM-DRW/CIFAR-10-2/'

datasetexample = np.load(cifarcdatadir + 'cifar10.1_v6_data.npy')
labelcifar = np.load(cifarcdatadir + 'cifar10.1_v6_labels.npy')

savefilename = 'predictions_val_2.csv'
    
val_datasetc = MyDataset(datasetexample, labelcifar, transform=transform_val)
val_loaderc = torch.utils.data.DataLoader(val_datasetc, batch_size=1, shuffle=False,
    num_workers=4, pin_memory=True)

model.eval()
logits = []
preds = []
targets = []
for i, (input, target) in enumerate(tqdm(val_loaderc)):
    input = input.cuda(gpu, non_blocking = True)
    logits_test = model(input)
    preds_test = F.softmax(logits_test, dim = 1)
    targets_test = F.one_hot(target, num_classes = num_classes)
    logits.append(logits_test.cpu().detach())
    preds.append(preds_test.cpu().detach())
    targets.append(targets_test)

logits = torch.cat(logits, dim=0)
preds = torch.cat(preds, dim=0)
targets = torch.cat(targets, dim=0)

df = pd.DataFrame(data=preds.numpy(), columns=cols_names_classes)
df_logits = pd.DataFrame(data=logits.numpy(), columns=cols_names_logits)
df_targets = pd.DataFrame(data=targets.numpy(), columns=cols_names_targets)
df = pd.concat([df, df_logits, df_targets], axis=1)
df.to_csv(os.path.join(cifarresultsdir, savefilename), index=False)

100%|█████████████████████████████████████████████████████████████████| 2000/2000 [00:11<00:00, 172.53it/s]


In [9]:
# test cifar-10-cv4
cifarresultsdir = '/vol/biomedic3/zl9518/ModelEvaluation/LDAM-DRW/cifar10results/'
cifarcdatadir = '/vol/biomedic3/zl9518/ModelEvaluation/LDAM-DRW/CIFAR-10-2v4/'

datasetexample = np.load(cifarcdatadir + 'cifar10.1_v4_data.npy')
labelcifar = np.load(cifarcdatadir + 'cifar10.1_v4_labels.npy')

savefilename = 'predictions_val_2v4.csv'
    
val_datasetc = MyDataset(datasetexample, labelcifar, transform=transform_val)
val_loaderc = torch.utils.data.DataLoader(val_datasetc, batch_size=1, shuffle=False,
    num_workers=4, pin_memory=True)

model.eval()
logits = []
preds = []
targets = []
for i, (input, target) in enumerate(tqdm(val_loaderc)):
    input = input.cuda(gpu, non_blocking = True)
    logits_test = model(input)
    preds_test = F.softmax(logits_test, dim = 1)
    targets_test = F.one_hot(target, num_classes = num_classes)
    logits.append(logits_test.cpu().detach())
    preds.append(preds_test.cpu().detach())
    targets.append(targets_test)

logits = torch.cat(logits, dim=0)
preds = torch.cat(preds, dim=0)
targets = torch.cat(targets, dim=0)

df = pd.DataFrame(data=preds.numpy(), columns=cols_names_classes)
df_logits = pd.DataFrame(data=logits.numpy(), columns=cols_names_logits)
df_targets = pd.DataFrame(data=targets.numpy(), columns=cols_names_targets)
df = pd.concat([df, df_logits, df_targets], axis=1)
df.to_csv(os.path.join(cifarresultsdir, savefilename), index=False)

100%|██████████████| 2021/2021 [00:10<00:00, 185.19it/s]
